---

> This is a Jupyter notebook builded is a python project with the code available on GitHub.
> (https://github.com/Anthony-Jhoiro/projet_ia_s1).
> Only the main functions will be presented in this file. If you want to see the implementation
> of those functions, I recommand checking directly the repository.

## Préparation des données
> Erratum
>
> Scikit ne supporte que les valeurs continuent pour les différents algorithmes utilisés. Par conséquent les champ sde catégories ont
> été passés en champs booléens puis les champs booléens sont passé en 0 et 1. C'est pourquoi nos retrouvons dans nos
> centre des valeurs continues pour des champs booléens.

In [25]:
from dataset_utils import add_score_g3_classes_columns, add_ratio_g1pg2_column

import csv_parser
import pandas as pd
import student_dataset as sd

# Define float output
pd.options.display.float_format = '{:,.4f}'.format

# Load DataFrame
mat_original = csv_parser.parse_student_csv("student-mat.csv", separator=";",
                                            cast_types=sd.dataset_types)
por_original = csv_parser.parse_student_csv("student-por.csv", separator=";",
                                            cast_types=sd.dataset_types)

# Create missing columns
add_score_g3_classes_columns(mat_original)
add_score_g3_classes_columns(por_original)
add_ratio_g1pg2_column(mat_original)
add_ratio_g1pg2_column(por_original)

mat_formatted = csv_parser.format_dataset(mat_original, sd.target_columns, sd.string_fields, sd.boolean_fields)
por_formatted = csv_parser.format_dataset(por_original, sd.target_columns, sd.string_fields, sd.boolean_fields)

## Partie 1 - Analyse des données

### Les écoles

__Les élèves de mathématique__

![Plot plots/plot_1_school.svg](used_plots/plot_school.mat.png)

__Les élèves de portugais__

![Plot plots/plot_1_school.svg](used_plots/plot_school.por.png)


L'attribut "school" nous informe l'école des élèves.
Nous pouvons constater que la grande majorité des élèves viennent de l'école Gabriel Pereira.
Les analyse seront donc bien plus influencées par les élèves de cette école et ne pourrait donc pas correspondre aux
élèves de l'autre école.

### L'adresse
__Les élèves de mathématique__

![Plot plots/plot_1_school.svg](used_plots/plot_address.mat.png)

__Les élèves de portugais__
![Plot plots/plot_1_school.svg](used_plots/plot_address.por.png)

L'attribut "address" nous renseigne sur le milieu dans lequel les élèves vivent. Ici, nous pouvons constatez que la
majorité des élèves vivent en milieu Urbain, que ce soit pour les cous de portugais ou les cours de mathématique.


### La situation des parents
__Les élèves de mathématique__

![Plot plots/plot_1_school.svg](used_plots/plot_Pstatus.mat.png)

__Les élèves de portugais__

![Plot plots/plot_1_school.svg](used_plots/plot_Pstatus.por.png)

L'attribut "Pstatus" prend pour valeur T si les parents de l'élève vivent ensemble et A dans les autres cas. La
majorité des parents des élèves vivent ensemble.

### Le sexe des élèves
__Les élèves de mathématique__

![Plot plots/plot_1_school.svg](used_plots/plot_sex.mat.png)

__Les élèves de portugais__

![Plot plots/plot_1_school.svg](used_plots/plot_sex.por.png)

L'attribut "sex" nous renseigne sur le sexe de l'élève. Ici, nous avons une majorité de fille parmi les élève ce qui est
conforme à la répartition en France cette année d'après cette publication de l'INSEE
(https://www.insee.fr/en/statistiques/2382597?sommaire=2382613).

### La note G3 (avec les classes)
__Les élèves de mathématique__

![Plot plots/plot_1_school.svg](used_plots/plot_score_G3_classes.mat.png)

__Les élèves de portugais__

![Plot plots/plot_1_school.svg](used_plots/plot_score_G3_classes.por.png)


Les notes sont globalement faibles dans les deux matières même si les notes des élèves de potugais sont généralement
plus élevées que celles des élèves de mathématique. (moins de F mais plus de D et de C).


## Partie 2 - Segmentation des étudiants

### Préparation du jeu de données
On retire les colonnes ciblées car nous souhaitons par la suite caractériser nos clusters par rapport à ces attributs.
Les attributs G1 et G2 sont également supprimés car l'attribut ratio_G1+G2 est obtenu par ces colonnes, ce qui fausse
donc le résultat.


In [26]:
excluded_columns = [
    "G1",
    "G2",
    "G3",
    "ratio_G1+G2",
    "score_G3_classes_A",
    "score_G3_classes_B",
    "score_G3_classes_C",
    "score_G3_classes_D",
    "score_G3_classes_F",
]

### Pour les élèves de mathématique

In [27]:
dataset_for_predictions = mat_formatted.copy()
dataset_for_predictions = csv_parser.drop_columns(dataset_for_predictions, excluded_columns)

# Nous souhaitons travailler avec 2 clusters à chaque fois.
cluster_count = 3

#### Algorithme du Kmean

En utilisant l'algorithme du knn sur notre jeu de donnée, nous optenons 3 clusters C0, C1 et C2 dont voici les centres :


In [28]:
from sklearn.cluster import KMeans


knn_predictions = KMeans(n_clusters=cluster_count, random_state=1)
knn_predictions.fit(dataset_for_predictions)

pd.DataFrame({
    "field": knn_predictions.feature_names_in_,
    "Centre de C0": knn_predictions.cluster_centers_[0],
    "Centre de C1": knn_predictions.cluster_centers_[1],
    "Centre de C2": knn_predictions.cluster_centers_[2],
})


,field,Centre de C0,Centre de C1,Centre de C2
0,age,16.5566,17.1605,17.8000
1,Medu,2.6731,3.0247,3.0000
2,Fedu,2.5081,2.5432,3.0000
3,failures,0.2977,0.4691,0.4000
4,schoolsup,0.1327,0.1111,0.2000
5,famsup,0.6052,0.6420,0.6000
6,paid,0.4563,0.4691,0.4000
7,activities,0.5178,0.4815,0.4000
8,nursery,0.7864,0.8272,0.8000
9,higher,0.9482,0.9630,0.8000


LE tableau ci dessus nous présente les différents attributs utilisés ainsi que leurs valeurs pour les centres de nos 3 clusters.

Interessons nous maintenant à la répartition des données au sein de ces clusters.

In [29]:
mat_original['knn_cluster'] = knn_predictions.labels_

knn_c0 = dataset_for_predictions[mat_original['knn_cluster'] == 0]
knn_c1 = dataset_for_predictions[mat_original['knn_cluster'] == 1]
knn_c2 = dataset_for_predictions[mat_original['knn_cluster'] == 2]


pd.DataFrame({
    "Cluster": ["C0", "C1", "C2"],
    "Nombre d'éléments": [len(knn_c0), len(knn_c1), len(knn_c2)],
})

,Cluster,Nombre d'éléments
0,C0,309
1,C1,81
2,C2,5


Le tableau ci dessus nous présente la quantité de donnée correspondant à chaque cluster.
Nous pouvons constater, que le cluster C0 est nettement plus imposant que les deux autres (il contient 98% de données).
Les prédictions résultantes en seront impacté.

#### Clustering hierarchique

Une autre méthode pour clusteriser nos données est le clustering hierarchique.

In [30]:
from sklearn import cluster

aglo_clusters = cluster.AgglomerativeClustering(cluster_count).fit(dataset_for_predictions)

mat_original['aglo_cluster'] = aglo_clusters.labels_

aglo_c0 = dataset_for_predictions[mat_original['aglo_cluster'] == 0]
aglo_c1 = dataset_for_predictions[mat_original['aglo_cluster'] == 1]
aglo_c2 = dataset_for_predictions[mat_original['aglo_cluster'] == 2]

aglo_c0_center = aglo_c0.mean(axis=0)
aglo_c1_center = aglo_c1.mean(axis=0)
aglo_c2_center = aglo_c2.mean(axis=0)

pd.DataFrame({
    "Centre de C0": aglo_c0_center,
    "Centre de C1": aglo_c1_center,
    "Centre de C2": aglo_c2_center,
})

,Centre de C0,Centre de C1,Centre de C2
age,16.5596,17.3175,17.8000
Medu,2.7095,2.9365,3.0000
Fedu,2.5138,2.5238,3.0000
failures,0.2844,0.5873,0.4000
schoolsup,0.1315,0.1111,0.2000
famsup,0.6055,0.6508,0.6000
paid,0.4587,0.4603,0.4000
activities,0.5168,0.4762,0.4000
nursery,0.7890,0.8254,0.8000
higher,0.9511,0.9524,0.8000


De la même manière, interessons nous à la répartition des données parmi nos clusters.

In [31]:
pd.DataFrame({
    "Cluster": ["C0", "C1", "C2"],
    "Nombre d'éléments": [len(aglo_c0), len(aglo_c1), len(aglo_c2)],
})

,Cluster,Nombre d'éléments
0,C0,327
1,C1,63
2,C2,5


Là encore, le cluster C0 est aussi très imposant. Une mauvaise répartition des données peut lourdement affecter



#### Evaluation de nos clusters

Nous allons maintenant analyser la qualité de nos clusters.

Il est peu pertinant de comparer les données des clusters car leur nombre d'éléments diffère trop.

### Pour les élèves de portugais

In [32]:
dataset_for_predictions = por_formatted.copy()
dataset_for_predictions = csv_parser.drop_columns(dataset_for_predictions, excluded_columns)

# Nous souhaitons travailler avec 2 clusters à chaque fois.
cluster_count = 3

#### Algorithme du Kmean

En utilisant l'algorithme du knn sur notre jeu de donnée, nous optenons 3 clusters C0, C1 et C2 dont voici les centres :


In [33]:
from sklearn.cluster import KMeans


knn_predictions = KMeans(n_clusters=cluster_count, random_state=1)
knn_predictions.fit(dataset_for_predictions)

pd.DataFrame({
    "field": knn_predictions.feature_names_in_,
    "Centre de C0": knn_predictions.cluster_centers_[0],
    "Centre de C1": knn_predictions.cluster_centers_[1],
    "Centre de C2": knn_predictions.cluster_centers_[2],
})


,field,Centre de C0,Centre de C1,Centre de C2
0,age,16.8414,17.1837,16.6273
1,Medu,2.5991,2.3061,2.4906
2,Fedu,2.3304,2.4082,2.2788
3,failures,0.2291,0.4694,0.1850
4,schoolsup,0.0881,0.0816,0.1180
5,famsup,0.6167,0.6327,0.6086
6,paid,0.0705,0.0204,0.0590
7,activities,0.5066,0.3878,0.4853
8,nursery,0.7665,0.7959,0.8257
9,higher,0.8987,0.7143,0.9142


Le tableau ci dessus nous montre les centres de nos clusters pour les élèves de portugais avec l'algorithme du Knn.
Nous pouvons constater qu'il y a peu de différences marquantes entre les différents centres ce qui peut être du à des données trop proches entre elles.


In [34]:
por_original['knn_cluster'] = knn_predictions.labels_

knn_c0 = dataset_for_predictions[por_original['knn_cluster'] == 0]
knn_c1 = dataset_for_predictions[por_original['knn_cluster'] == 1]
knn_c2 = dataset_for_predictions[por_original['knn_cluster'] == 2]


pd.DataFrame({
    "Cluster": ["C0", "C1", "C2"],
    "Nombre d'éléments": [len(knn_c0), len(knn_c1), len(knn_c2)],
})

,Cluster,Nombre d'éléments
0,C0,227
1,C1,49
2,C2,373


Les données sont bien mieux réparties dans les clusters par rapport à la méthode du Knn.
Cependant, il y a toujours un grand écart entre le plus petit cluster (C1) et le plus grand (C2).

#### Clustering hierarchique

Une autre méthode pour clusteriser nos données est le clustering hierarchique.

In [35]:
from sklearn import cluster

aglo_clusters = cluster.AgglomerativeClustering(cluster_count).fit(dataset_for_predictions)

por_original['aglo_cluster'] = aglo_clusters.labels_

aglo_c0 = dataset_for_predictions[por_original['aglo_cluster'] == 0]
aglo_c1 = dataset_for_predictions[por_original['aglo_cluster'] == 1]
aglo_c2 = dataset_for_predictions[por_original['aglo_cluster'] == 2]

aglo_c0_center = aglo_c0.mean(axis=0)
aglo_c1_center = aglo_c1.mean(axis=0)
aglo_c2_center = aglo_c2.mean(axis=0)

pd.DataFrame({
    "Centre de C0": aglo_c0_center,
    "Centre de C1": aglo_c1_center,
    "Centre de C2": aglo_c2_center,
})

,Centre de C0,Centre de C1,Centre de C2
age,16.6401,17.2414,17.1132
Medu,2.5331,2.6552,2.3868
Fedu,2.3035,2.6207,2.2358
failures,0.1926,0.4138,0.3113
schoolsup,0.1148,0.0345,0.0755
famsup,0.6051,0.6207,0.6509
paid,0.0661,0.0345,0.0377
activities,0.4883,0.2759,0.5283
nursery,0.8016,0.7931,0.8113
higher,0.9125,0.7586,0.8396


De la même manière, interessons nous à la répartition des données parmi nos clusters.

In [36]:
pd.DataFrame({
    "Cluster": ["C0", "C1", "C2"],
    "Nombre d'éléments": [len(aglo_c0), len(aglo_c1), len(aglo_c2)],
})

,Cluster,Nombre d'éléments
0,C0,514
1,C1,29
2,C2,106


Les données sont ici très mal réparties entre les clusters, le cluster C0 contenant 79% des données.
Les analyses résultantes seront lourdement impactés et une donnée aura bien plus de chance d'être associée à ce cluster plutôt qu'aux autres.


## Evaluation de nos clusters

Nous allons maintenant analyser la qualité de nos clusters.



In [37]:

from sklearn import metrics

pd.DataFrame({
    "Metric": ["G3 : Homogeneity Coefficient", "G3 : Silhouette Coefficient", "G1+G2 : Homogeneity Coefficient",
               "G1+G2 : Silhouette Coefficient"],
    "Valeur avec méthode Knn": [
        metrics.homogeneity_score(por_original['knn_cluster'], por_original['G3']),
        metrics.silhouette_score(dataset_for_predictions, por_original['G3']),
        metrics.homogeneity_score(por_original['knn_cluster'], por_original['ratio_G1+G2']),
        metrics.silhouette_score(dataset_for_predictions, por_original['ratio_G1+G2'])
    ],
    "Valeur avec méthode hierarchique": [
        metrics.homogeneity_score(por_original['aglo_cluster'], por_original['G3']),
        metrics.silhouette_score(dataset_for_predictions, por_original['G3']),
        metrics.homogeneity_score(por_original['aglo_cluster'], por_original['ratio_G1+G2']),
        metrics.silhouette_score(dataset_for_predictions, por_original['ratio_G1+G2'])
    ],
})

,Metric,Valeur avec méthode Knn,Valeur avec méthode hierarchique
0,G3 : Homogeneity Coefficient,0.0654,0.0652
1,G3 : Silhouette Coefficient,-0.1470,-0.1470
2,G1+G2 : Homogeneity Coefficient,0.0667,0.0680
3,G1+G2 : Silhouette Coefficient,-0.2302,-0.2302


Les résultats sont très similaires avec nos deux méthodes.

Le score d'homogénéité est très mauvais (trop proche de zéro) avec nos deux méthodes de clustering. Nous pouvons donc en déduire que :
- Il est possible qu'il y ai eu une erreur dans la manipulation des données
- Les élèves sont trop différents dans les jeux de données et que les clusters ne reflètent donc pas la réalité

La silhouette est très mauvaise aussi (inférieur à zéro) ce qui indique que les éléves ne sont pas bien placés dans les différents clusters.

## Partie 3 - Prédiction

In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


### Pour G3
#### Pour les élèves de math

In [39]:
train_data, test_data = train_test_split(mat_formatted, test_size=0.3)

train_data_g3= train_data["G3"]
train_data_for_g3 = train_data.drop(labels=["G3"], axis=1)
test_data_g3 = test_data["G3"]
test_data_for_g3 = test_data.drop(labels=["G3"], axis=1)


##### Méthode Kmean

In [40]:
knn = KNeighborsClassifier()
knn.fit(train_data_for_g3, train_data_g3)
predictions = knn.predict(test_data_for_g3)

knn_absolute_errors = np.abs(np.subtract(predictions, test_data_g3))
knn_square_errors = np.square(knn_absolute_errors)

knn_mse = np.average(knn_square_errors)
knn_rmse = np.sqrt(knn_mse)
knn_mae = np.average(knn_absolute_errors)

##### Linear regression

In [41]:
from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression().fit(train_data_for_g3, train_data_g3)
lr_results = lr_reg.predict(test_data_for_g3)

lr_absolute_errors = np.abs(np.subtract(lr_results, test_data_g3))
lr_square_errors = np.square(lr_absolute_errors)

lr_mse = np.average(lr_square_errors)
lr_rmse = np.sqrt(lr_mse)
lr_mae = np.average(lr_absolute_errors)

##### Resultats

In [42]:
pd.DataFrame({
    "Metriques": ["MAE", "MSE", "RMSE"],
    "Valeurs pour Knn": [knn_mae, knn_mse, knn_rmse],
    "Valeurs pour Regression linéaire": [lr_mae, lr_mse, lr_rmse],
})

,Metriques,Valeurs pour Knn,Valeurs pour Regression linéaire
0,MAE,1.2437,1.4478
1,MSE,5.9328,4.7874
2,RMSE,2.4357,2.1880


Le tableau ci dessous présentent les valuers des metriques MSE, RMSE et MAE pour les prédictions des algorithmes de regression linéaire.
Ces différentes métriques se basent sur les erreurs des différents modes de prédiction.
Plus une erreur est proche de zéro, plus la prédictionest juste. A l'inverse, plus l'erreur est grande, plus les erreurs de prédiction sont grandes.

Nous pouvons donc constater que l'algorithme de regression linéaire est bien meilleur que celui du kmean pour calculer les notes des élèves sur notre jeu de données.

#### Pour les élèves de portugais

In [43]:
train_data, test_data = train_test_split(por_formatted, test_size=0.3)

train_data_g3 = train_data["G3"]
train_data_for_g3 = train_data.drop(labels=["G3"], axis=1)
test_data_g3 = test_data["G3"]
test_data_for_g3 = test_data.drop(labels=["G3"], axis=1)


##### Méthode Kmean

In [44]:
knn = KNeighborsClassifier()
knn.fit(train_data_for_g3, train_data_g3)
predictions = knn.predict(test_data_for_g3)

knn_absolute_errors = np.abs(np.subtract(predictions, test_data_g3))
knn_square_errors = np.square(knn_absolute_errors)

knn_mse = np.average(knn_square_errors)
knn_rmse = np.sqrt(knn_mse)
knn_mae = np.average(knn_absolute_errors)

##### Linear regression

In [45]:
from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression().fit(train_data_for_g3, train_data_g3)
lr_results = lr_reg.predict(test_data_for_g3)

lr_absolute_errors = np.abs(np.subtract(lr_results, test_data_g3))
lr_square_errors = np.square(lr_absolute_errors)

lr_mse = np.average(lr_square_errors)
lr_rmse = np.sqrt(lr_mse)
lr_mae = np.average(lr_absolute_errors)

##### Resultats

In [46]:
pd.DataFrame({
    "Metriques": ["MAE", "MSE", "RMSE"],
    "Valeurs pour Knn": [knn_mae, knn_mse, knn_rmse],
    "Valeurs pour Regression linéaire": [lr_mae, lr_mse, lr_rmse],
})

,Metriques,Valeurs pour Knn,Valeurs pour Regression linéaire
0,MAE,0.7590,0.6319
1,MSE,1.3846,0.7015
2,RMSE,1.1767,0.8375


Nous pouvons constater une nouvelle fois que l'algorithme de regression linéaire est bien meilleur que celui du kmean pour calculer les notes des élèves sur notre jeu de données.


### Pour ratio_G1+G2
#### Pour les élèves de math

In [47]:
train_data, test_data = train_test_split(mat_formatted, test_size=0.3)

train_data_g3 = train_data["ratio_G1+G2"]
train_data_for_g3 = train_data.drop(labels=["ratio_G1+G2", "G1", "G2"], axis=1)
test_data_g3 = test_data["ratio_G1+G2"]
test_data_for_g3 = test_data.drop(labels=["ratio_G1+G2", "G1", "G2"], axis=1)


##### Méthode Kmean

In [48]:
knn = KNeighborsClassifier()
knn.fit(train_data_for_g3, train_data_g3)
predictions = knn.predict(test_data_for_g3)

knn_absolute_errors = np.abs(np.subtract(predictions, test_data_g3))
knn_square_errors = np.square(knn_absolute_errors)

knn_mse = np.average(knn_square_errors)
knn_rmse = np.sqrt(knn_mse)
knn_mae = np.average(knn_absolute_errors)

##### Linear regression

In [49]:
from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression().fit(train_data_for_g3, train_data_g3)
lr_results = lr_reg.predict(test_data_for_g3)

lr_absolute_errors = np.abs(np.subtract(lr_results, test_data_g3))
lr_square_errors = np.square(lr_absolute_errors)

lr_mse = np.average(lr_square_errors)
lr_rmse = np.sqrt(lr_mse)
lr_mae = np.average(lr_absolute_errors)

##### Resultats

In [50]:
pd.DataFrame({
    "Metriques": ["MAE", "MSE", "RMSE"],
    "Valeurs pour Knn": [knn_mae, knn_mse, knn_rmse],
    "Valeurs pour Regression linéaire": [lr_mae, lr_mse, lr_rmse],
})

,Metriques,Valeurs pour Knn,Valeurs pour Regression linéaire
0,MAE,3.5126,2.1168
1,MSE,19.4454,7.1094
2,RMSE,4.4097,2.6663


Le tableau ci dessous présentent les valuers des metriques MSE, RMSE et MAE pour les prédictions des algorithmes de regression linéaire.
Ces différentes métriques se basent sur les erreurs des différents modes de prédiction.
Plus une erreur est proche de zéro, plus la prédictionest juste. A l'inverse, plus l'erreur est grande, plus les erreurs de prédiction sont grandes.

Nous pouvons donc constater que l'algorithme de regression linéaire est bien meilleur que celui du kmean pour calculer les notes des élèves sur notre jeu de données.

#### Pour les élèves de portugais

In [51]:
train_data, test_data = train_test_split(por_formatted, test_size=0.3)

train_data_g3 = train_data["ratio_G1+G2"]
train_data_for_g3 = train_data.drop(labels=["ratio_G1+G2", "G1", "G2"], axis=1)
test_data_g3 = test_data["ratio_G1+G2"]
test_data_for_g3 = test_data.drop(labels=["ratio_G1+G2", "G1", "G2"], axis=1)


##### Méthode Kmean

In [52]:
knn = KNeighborsClassifier()
knn.fit(train_data_for_g3, train_data_g3)
predictions = knn.predict(test_data_for_g3)

knn_absolute_errors = np.abs(np.subtract(predictions, test_data_g3))
knn_square_errors = np.square(knn_absolute_errors)

knn_mse = np.average(knn_square_errors)
knn_rmse = np.sqrt(knn_mse)
knn_mae = np.average(knn_absolute_errors)

##### Linear regression

In [53]:
from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression().fit(train_data_for_g3, train_data_g3)
lr_results = lr_reg.predict(test_data_for_g3)

lr_absolute_errors = np.abs(np.subtract(lr_results, test_data_g3))
lr_square_errors = np.square(lr_absolute_errors)

lr_mse = np.average(lr_square_errors)
lr_rmse = np.sqrt(lr_mse)
lr_mae = np.average(lr_absolute_errors)

##### Resultats

In [54]:
pd.DataFrame({
    "Metriques": ["MAE", "MSE", "RMSE"],
    "Valeurs pour Knn": [knn_mae, knn_mse, knn_rmse],
    "Valeurs pour Regression linéaire": [lr_mae, lr_mse, lr_rmse],
})

,Metriques,Valeurs pour Knn,Valeurs pour Regression linéaire
0,MAE,2.5487,1.7521
1,MSE,11.3692,5.2105
2,RMSE,3.3718,2.2827


Nous pouvons constater une nouvelle fois que l'algorithme de regression linéaire est bien meilleur que celui du kmean pour calculer les notes des élèves sur notre jeu de données.

## Conclusion

Pour chacune de nos situations, la méthode de regression linéaire donne des erreurs plus faibles que l'alagithme du Knn. Cette méthode est donc à privilégier.